# Measure Pitch, HNR, Jitter, Shimmer, Formants, and Estimate VTL

## Import the external modules

In [2]:
#!/usr/bin/env python3
import glob
import numpy as np
import pandas as pd
import parselmouth 
import statistics


from parselmouth.praat import call
from scipy.stats.mstats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
path_to_wav_files = '../Wegner'

## This function measures duration, pitch, HNR, jitter, and shimmer

In [4]:
# This is the function to measure source acoustics using default male parameters.

def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    duration = call(sound, "Get total duration") # duration
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    
    return duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

## This function measures formants at each glottal pulse

Puts, D. A., Apicella, C. L., & Cárdenas, R. A. (2012). Masculine voices signal men's threat potential in forager and industrial societies. Proceedings of the Royal Society of London B: Biological Sciences, 279(1728), 601-609.

Adapted from: DOI 10.17605/OSF.IO/K2BHS

In [5]:
# This function measures formants using Formant Position formula
def measureFormants(sound, wave_file, f0min,f0max):
    sound = parselmouth.Sound(sound) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    f1_mean = statistics.mean(f1_list)
    f2_mean = statistics.mean(f2_list)
    f3_mean = statistics.mean(f3_list)
    f4_mean = statistics.mean(f4_list)
    
    # calculate median formants across pulses, this is what is used in all subsequent calcualtions
    # you can use mean if you want, just edit the code in the boxes below to replace median with mean
    f1_median = statistics.median(f1_list)
    f2_median = statistics.median(f2_list)
    f3_median = statistics.median(f3_list)
    f4_median = statistics.median(f4_list)
    
    return f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median


## This function runs a 2-factor Principle Components Analysis (PCA) on Jitter and Shimmer

In [6]:
def runPCA(df):
    # z-score the Jitter and Shimmer measurements
    measures = ['localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter',
                'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer']
    x = df.loc[:, measures].values
    x = StandardScaler().fit_transform(x)
    # PCA
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['JitterPCA', 'ShimmerPCA'])
    principalDf
    return principalDf
# kann wegbleiben

## This block of code runs the above functions on all of the '.wav' files in the /audio folder

In [10]:
# create lists to put the results
file_list = []
duration_list = []
mean_F0_list = []
sd_F0_list = []
hnr_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
ddpJitter_list = []
localShimmer_list = []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list = []
ddaShimmer_list = []
f1_mean_list = []
f2_mean_list = []
f3_mean_list = []
f4_mean_list = []
f1_median_list = []
f2_median_list = []
f3_median_list = []
f4_median_list = []

# Go through all the wave files in the folder and measure all the acoustics
for wave_file in glob.glob(path_to_wav_files+"/*.wav"):
    sound = parselmouth.Sound(wave_file)
    (duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, 
     localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(
        sound, 75, 300, "Hertz")
    (f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median) = measureFormants(
        sound, wave_file, 75, 300)
    file_list.append(wave_file) # make an ID list
    duration_list.append(duration) # make duration list
    mean_F0_list.append(meanF0) # make a mean F0 list
    sd_F0_list.append(stdevF0) # make a sd F0 list
    hnr_list.append(hnr) #add HNR data
    
    # add raw jitter and shimmer measures
    localJitter_list.append(localJitter)
    localabsoluteJitter_list.append(localabsoluteJitter)
    rapJitter_list.append(rapJitter)
    ppq5Jitter_list.append(ppq5Jitter)
    ddpJitter_list.append(ddpJitter)
    localShimmer_list.append(localShimmer)
    localdbShimmer_list.append(localdbShimmer)
    apq3Shimmer_list.append(apq3Shimmer)
    aqpq5Shimmer_list.append(aqpq5Shimmer)
    apq11Shimmer_list.append(apq11Shimmer)
    ddaShimmer_list.append(ddaShimmer)
    
    # add the formant data
    f1_mean_list.append(f1_mean)
    f2_mean_list.append(f2_mean)
    f3_mean_list.append(f3_mean)
    f4_mean_list.append(f4_mean)
    f1_median_list.append(f1_median)
    f2_median_list.append(f2_median)
    f3_median_list.append(f3_median)
    f4_median_list.append(f4_median)

In [11]:
f1_mean_list
#nur zur Überpüfung

[514.4937605662323,
 567.9794008445031,
 364.18671524755604,
 529.6783610136556,
 563.8404430969476,
 525.6150612131795,
 645.0168658743127,
 444.4163685138116,
 488.9042685103732,
 474.9345923795171,
 484.3353371348433,
 578.8954751312931,
 455.7063514957035,
 313.93918753898,
 600.6329933077751,
 550.5747814553291,
 530.9870440643971,
 512.8224724041394,
 511.3785689591542,
 458.2774722101212,
 520.8657390102544,
 516.325315377039,
 429.7035731920803,
 462.0036192647773,
 397.147709391028,
 547.383066211452,
 453.46007230003124,
 505.23768682528583,
 620.9820140797511,
 567.7116001790823,
 541.835507653638,
 479.5437936538365,
 340.8853589274669,
 490.014739160713,
 571.6845999056534,
 533.0934657081328,
 536.6397642319874,
 457.50257557898,
 488.0163107663581,
 521.1310013922161,
 519.4452194597113]

## This block of code adds all of that data we just generated to a Pandas data frame

In [12]:
# Add the data to Pandas
df = pd.DataFrame(np.column_stack([file_list, duration_list, mean_F0_list, sd_F0_list, hnr_list, 
                                   localJitter_list, localabsoluteJitter_list, rapJitter_list, 
                                   ppq5Jitter_list, ddpJitter_list, localShimmer_list, 
                                   localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, 
                                   apq11Shimmer_list, ddaShimmer_list, f1_mean_list, 
                                   f2_mean_list, f3_mean_list, f4_mean_list, 
                                   f1_median_list, f2_median_list, f3_median_list, 
                                   f4_median_list]),
                                   columns=['voiceID', 'duration', 'meanF0Hz', 'stdevF0Hz', 'HNR', 
                                            'localJitter', 'localabsoluteJitter', 'rapJitter', 
                                            'ppq5Jitter', 'ddpJitter', 'localShimmer', 
                                            'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 
                                            'apq11Shimmer', 'ddaShimmer', 'f1_mean', 'f2_mean', 
                                            'f3_mean', 'f4_mean', 'f1_median', 
                                            'f2_median', 'f3_median', 'f4_median'])

pcaData = runPCA(df) # Run jitter and shimmer PCA
df = pd.concat([df, pcaData], axis=1) # Add PCA data
# reload the data so it's all numbers
df.to_csv("processed_results.csv", index=False)
df = pd.read_csv('processed_results.csv', header=0)
df.sort_values('voiceID').head(20)

,voiceID,duration,meanF0Hz,stdevF0Hz,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,...,f1_mean,f2_mean,f3_mean,f4_mean,f1_median,f2_median,f3_median,f4_median,JitterPCA,ShimmerPCA
0,../Wegner\Wegner_16_seg_00000000.wav,6.69,210.652453,30.909104,12.388043,0.030422,0.000145,0.014803,0.017130,0.044408,...,514.493761,1687.033285,2843.755036,3822.966924,392.422237,1732.831674,3010.805058,3894.497065,-1.277184,-1.111271
1,../Wegner\Wegner_16_seg_00000001.wav,1.78,219.365428,32.645018,16.261678,0.015167,0.000069,0.007675,0.008471,0.023026,...,567.979401,1483.325442,2890.441545,3874.150362,400.316936,1523.582835,3030.494195,3948.166902,4.456632,2.051313
2,../Wegner\Wegner_16_seg_00000002.wav,2.92,227.286451,47.279022,19.494601,0.015788,0.000069,0.007040,0.009223,0.021121,...,364.186715,1430.888424,2788.914754,3675.516415,303.706591,1140.081962,3002.890676,3734.642377,8.177602,-1.352489
3,../Wegner\Wegner_16_seg_00000003.wav,6.13,204.341022,36.597091,12.098336,0.028401,0.000139,0.014262,0.016671,0.042785,...,529.678361,1727.370313,2897.140471,3908.300151,426.373583,1805.499622,2955.691695,4014.538921,-1.203342,-0.559052
4,../Wegner\Wegner_16_seg_00000004.wav,4.38,209.720927,40.567627,12.044526,0.026627,0.000127,0.013704,0.016141,0.041112,...,563.840443,1835.488709,2955.337746,3969.420132,508.920057,1899.662846,3005.825401,4022.434247,-1.060610,0.389843
5,../Wegner\Wegner_16_seg_00000005.wav,3.72,191.486545,27.285377,10.935678,0.019016,0.000100,0.009134,0.011152,0.027401,...,525.615061,1725.812069,2809.227734,3755.764871,469.884293,1734.906229,2896.052808,3835.268770,0.701470,3.274245
6,../Wegner\Wegner_16_seg_00000006.wav,2.46,212.565695,35.140675,9.039244,0.037535,0.000178,0.021433,0.021781,0.064300,...,645.016866,1681.869200,2818.545964,3803.466429,604.141217,1669.314858,3029.724355,3862.213157,-5.538222,-2.892599
7,../Wegner\Wegner_16_seg_00000007.wav,3.78,190.471188,33.783368,12.781051,0.027049,0.000142,0.014122,0.015844,0.042366,...,444.416369,1906.112984,2836.114385,3737.400534,369.986043,1917.168757,2942.116840,3839.862485,-0.901388,-0.240007
8,../Wegner\Wegner_16_seg_00000008.wav,3.98,182.604261,36.200272,14.290371,0.023152,0.000126,0.011180,0.012603,0.033540,...,488.904269,1663.284012,2741.263645,3717.721749,359.088686,1702.949745,2928.543810,3827.323073,1.166869,0.648207
9,../Wegner\Wegner_16_seg_00000009.wav,2.70,217.556155,32.396695,9.946097,0.027095,0.000125,0.011853,0.013930,0.035558,...,474.934592,1721.344257,2877.467841,3767.714087,390.986136,1780.400207,3005.249823,3841.438111,-1.126336,1.706214


## Next we calculate the vocal-tract length estimates

### Formant position
 Puts, D. A., Apicella, C. L., & Cárdenas, R. A. (2012). Masculine voices signal men's threat potential in forager and industrial societies. Proceedings of the Royal Society of London B: Biological Sciences, 279(1728), 601-609.

In [13]:
df['pF'] = (zscore(df.f1_median) + zscore(df.f2_median) + zscore(df.f3_median) + zscore(df.f4_median)) / 4

### Formant Dispersion
Fitch, W. T. (1997). Vocal tract length and formant frequency dispersion correlate with body size in rhesus macaques. The Journal of the Acoustical Society of America, 102(2), 1213-1222.

In [14]:
df['fdisp'] = (df['f4_median'] - df['f1_median']) / 3

### Fn (Average Formant)
Pisanski, K., & Rendall, D. (2011). The prioritization of voice fundamental frequency or formants in listeners’ assessments of speaker size, masculinity, and attractiveness. The Journal of the Acoustical Society of America, 129(4), 2201-2212.

In [15]:
df['avgFormant'] = (df['f1_median'] + df['f2_median'] + df['f3_median'] + df['f4_median']) / 4

### MFF 
Smith, D. R., & Patterson, R. D. (2005). The interaction of glottal-pulse rate and vocal-tract length in judgements of speaker size, sex, and age. The Journal of the Acoustical Society of America, 118(5), 3177-3186.

In [16]:
df['mff'] = (df['f1_median'] * df['f2_median'] * df['f3_median'] * df['f4_median']) ** 0.25

### Fitch VTL
Fitch, W. T. (1997). Vocal tract length and formant frequency dispersion correlate with body size in rhesus macaques. The Journal of the Acoustical Society of America, 102(2), 1213-1222.

In [17]:
# reload the data again
df.to_csv("processed_results.csv", index=False)
df = pd.read_csv('processed_results.csv', header=0)

df['fitch_vtl'] = ((1 * (35000 / (4 * df['f1_median']))) +
                   (3 * (35000 / (4 * df['f2_median']))) + 
                   (5 * (35000 / (4 * df['f3_median']))) + 
                   (7 * (35000 / (4 * df['f4_median'])))) / 4

### $\Delta$F 
Reby,D.,& McComb,K.(2003). Anatomical constraints generate honesty: acoustic cues to age and weight in the roars of red deer stags. Animal Behaviour, 65, 519e-530.

In [23]:
xysum = (0.5 * df['f1_median']) + (1.5 * df['f2_median']) + (2.5 * df['f3_median']) + (3.5 * df['f4_median'])
xsquaredsum = (0.5 * 2) + (1.5 * 2) + (2.5 * 2) + (3.5 * 2)
df['delta_f'] = xysum / xsquaredsum

### VTL($\Delta$F)
Reby,D.,&McComb,K.(2003).Anatomical constraints generate honesty: acoustic cues to age and weight in the roars of red deer stags. Animal Behaviour, 65, 519e-530.

In [24]:
df['vtl_delta_f'] = 35000 / (2 * df['delta_f'])

## Save the final data

In [25]:
# Write out the final dataframe
df.to_csv("processed_results2.csv", index=False)

## Run this to tell you when it's done

In [26]:
print("finished")

finished


In [27]:
df.to_pickle('..\df_werte.pkl')